In [5]:
df.shape

(0, 20)

In [10]:
import pandas  as pd
df = pd.read_csv(r"C:\Users\trilo\Downloads\MLOps\End_To_End_DATA_DRITF_Pipeline\data\processed\Preprocessed_Data.csv")
print(df.shape)  # Should show nonzero rows
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Drop rows only if Churn or TotalCharges is missing
df = df.dropna(subset=['TotalCharges'])

print(df.shape)  # Should show nonzero rows

(7032, 20)
(7032, 20)


In [7]:
df.shape

(0, 20)